In [4]:
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 4)
pd.set_option("display.max_columns", 4)

In [5]:
df = pd.read_csv('Int_papers_scopus2017.xml', header=0)
#Affiliationsの列だけを読み出す
df_header_usecols = pd.read_csv('Int_papers_scopus2017.xml', usecols=['Affiliations'])

#Affiliationsが；で区切られているのでそこでsplitする
#print(df['Affiliations'].str.split(';', expand=True))
#それをデータフレームにする
#pd.concat()で連結（結合）するときに必要な列（Affiliations）だけ選択

In [6]:
df3 = pd.concat([df['Affiliations'].str.split(';', expand=True)], axis=1)

#行数を取得=論文の数
print(len(df3))

#列数を取得
print(len(df3.columns))
#機関数が一番多いところの列数になる

618
58


#カラムじゃないとやりづらかったので、入れ替え
#行と列を入れ替えたデータフレーム (列カラムが各論文の機関名)

In [8]:
df4 = df3.T

#列ごとに（各論文）ごとに欠損値でない要素の個数をカウント（=機関数）
print(df4.count())

#次は、このうち何個Japanを含むかを計算

0       2
1       5
       ..
616    15
617    28
Length: 618, dtype: int64


In [10]:
df10 = pd.DataFrame(df4.count())
#カラムに名前を付ける
df10.columns = ['Affiliations']

In [11]:
#各論文（各カラム）の中に'Japan’所属の機関の数がいくつあるか
#Japanの機関数をdf5とする
#python pandas forループの計算結果をデータフレームに入れる方法がわからなくて検索したところ、
#一度numpy配列に入れてからデータフレームに変換することでできるとの方法を見つけてやってみる　→できた

x = np.zeros(len(df4.columns), dtype=np.int64)  #データ型をintに指定
for i in range(len(df4.columns)):
        df5 = df4[i].str.contains('Japan').sum()
        x[i] = df5
df5 = pd.DataFrame(x)
df5.columns = ['Japan']
#各論文（各カラム）の機関数=AffiliationNumberと比較して、'Japan'所属の機関数と等しければ国内論文、数が異なれば国際共著論文と判断する

In [12]:
#各論文の機関の数（df10)と日本機関の数(df5)を一つのデータフレームにする
df11 = pd.concat([df10, df5], axis=1)
print(df11)

     Affiliations  Japan
0               2      2
1               5      5
..            ...    ...
616            15      8
617            28     28

[618 rows x 2 columns]


In [13]:
#機関の数と日本機関の数が等しくないところをTrue=Internationalとする
df11['International'] = (df11['Affiliations']!=df11['Japan'])
df11['International']

0      False
1      False
       ...  
616     True
617    False
Name: International, Length: 618, dtype: bool

In [20]:
print('この年のscopus論文の総数は')
print(len(df4.columns))

International = df11['International']==True
print('国際共著論文は')
print(International.sum())

Domestic = df11['International']==False
print('日本のみの論文数は')
print(Domestic.sum())

この年のscopus論文の総数は
618
国際共著論文は
126
日本のみの論文数は
492
